In [3]:

import torch
import os
import torchvision
from torch import nn as nn
from torch import optim
from torch.utils import data 
from torchvision import transforms
from torchvision import models
from torchvision import datasets 
import matplotlib.pyplot as plt
import numpy as np
import time as t
from tqdm import tqdm
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score


In [4]:
#图片预处理
train_trans=transforms.Compose([
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
#RCTN
test_trans=transforms.Compose([
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
#读取图片
dataset_path='../input/fruit30/fruit30_split'
train_path=os.path.join(dataset_path,"train")
val_path=os.path.join(dataset_path,"val")
train_data=datasets.ImageFolder(train_path,train_trans)
val_data=datasets.ImageFolder(val_path,test_trans)
train_iter=data.DataLoader(train_data,shuffle=True,batch_size=64)
val_iter=data.DataLoader(val_data,shuffle=True,batch_size=64)
# #可视化图片
# images,labels=next(iter(train_loader))
# images=images.numpy()
# images[63].shape
# plt.hist(images[2].flatten(),bins=6)
# plt.show()
#训练模型
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#迁移学习
model=models.vgg16(pretrained=True)
model.fc=nn.Linear(model.fc.in_features,30)
updater=optim.Adam(model.fc.parameters())
lr_scheduler=lr_scheduler.StepLR(updater,step_size=5,gamma=0.5)
loss=nn.CrossEntropyLoss()
epochs=100
# viz = Visdom()
# # 绘制起点
# viz.line([0.], [0.], win="train loss", opts=dict(title='train_loss'))
train_l=[]
epoch_x=[]
train_real=0
train_total=0
#格式转换函数 
def time_change(time):
       new_time=t.localtime(time)
       otherStyleTime = t.strftime("%Hh%Mm%Ss", new_time)
       return   otherStyleTime          
t_start=t.time()
model.train()
model.to(device)
print("training....")
for epoch in range(epochs):
       for img,labels in tqdm(train_iter):
           img,labels=img.to(device),labels.to(device)
           updater.zero_grad()
           l=loss(model(img),labels)
           l.backward()
           updater.step()
            #每批训练后的测试
           _,pred=torch.max(model(img),1)
           pred=pred.cpu().numpy()
           labels=labels.cpu().numpy()
           train_acc=accuracy_score(pred,labels)
       lr_scheduler.step() 
       epoch_x.append(epoch+1)
       train_l.append(l.item())    
       t_end=t.time()
       print(f"进度{epoch+1}/{epochs}")
       print(f'epoch:{epoch+1},train_loss:{l.item():.3f},train_acc:{train_acc.item():.3f},time:{time_change(t_end-t_start)}')
plt.title("class of images") 
plt.xlabel("epoch") 
plt.ylabel("loss")
plt.plot(epoch_x,train_l)
plt.grid()
plt.legend()
plt.show()


In [5]:

#模型评估
model.eval()
with torch.no_grad():
          correct=0
          total=0
          for img,labels in val_iter:
                img,labels=img.to(device),labels.to(device)
                output=model(img)  
                _, preds = torch.max(output, 1)
                total += labels.size(0)
                correct += (preds == labels).sum()
          test_acc=100*correct/total
          print(f"测试集的模型评估精度为{test_acc:.2f}% ")    
                  
#模型保存
torch.save(model,"Image classification_resnet18.pth") 
print("save succcess!")